In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import fu, nu
from pandas import DataFrame, Index, Series, concat, notnull, to_datetime
import json
import numpy as np
import os
import os.path as osp
import pandas as pd
import re

In [3]:

# load data frames
data_frames_dict = nu.load_data_frames(metrics_evaluation_open_world_df='', first_responder_master_registry_file_stats_df='')
logs_df = data_frames_dict['metrics_evaluation_open_world_df']
print(logs_df.shape) # (171766, 111)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_file_stats_df.pkl.
(171766, 124)



# JSON Stats Created for Metrics Evaluation Open World

In [4]:

# Get all the Open World JSON Stats into one data frame
metrics_evaluation_open_world_json_stats_df = DataFrame([])
def get_directory_paths(participant_id, logs_path='../data/logs/Metrics Evaluation Open World'):
    
    # Get all the directory names which are suffixed by the participant ID
    directories_list = os.listdir(logs_path)
    
    # Loop through the directories and IDs and find a pair that matches
    participant_dirs_list = [dir_name for dir_name in directories_list if dir_name.endswith(str(participant_id))]

    return participant_dirs_list
logs_path = '../data/logs/Human_Sim_Metrics_Data 4-12-2024'
for participant_id in range(2_024_201, 2_024_223+1):
    for dir_name in get_directory_paths(participant_id, logs_path):
        
        # Add the JSONs to the data frame
        folder_path = osp.join(logs_path, dir_name)
        
        # Iterate over the files in the current subdirectory
        for file_name in os.listdir(folder_path):
            
            # If the file is a JSON file
            if file_name.endswith('.json'):

                # Create a data frame from the flattened dictionary
                json_path = osp.join(folder_path, file_name)
                with open(json_path, 'r') as f: file_json = json.load(f)
                row_dict = {
                    'participant_id': participant_id,
                    'sub_directory': folder_path.split('/')[-1],
                    'json_file_name': file_name
                }
                flattened_json_dict = nu.get_row_dictionary(file_json, row_dict=row_dict, key_prefix='')
                
                # You've got to clean the Session IDs
                session_uuid = flattened_json_dict.pop('sessionId')
                if session_uuid.endswith('_'): session_uuid = session_uuid[:-1]
                flattened_json_dict['session_uuid'] = session_uuid
                json_stats_df = DataFrame(flattened_json_dict, index=Index([0]))
                
                # Append the data frame for the current subdirectory to the main data frame and break the participant ID loop
                metrics_evaluation_open_world_json_stats_df = pd.concat([metrics_evaluation_open_world_json_stats_df, json_stats_df], axis='index')

metrics_evaluation_open_world_json_stats_df = metrics_evaluation_open_world_json_stats_df.reset_index(drop=True)
nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
print(metrics_evaluation_open_world_json_stats_df.participant_id.nunique()) # 22
print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3525)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv
22
(43, 3525)


In [5]:

metrics_evaluation_open_world_json_stats_df = nu.load_object('metrics_evaluation_open_world_json_stats_df')
if 'logger_version' not in metrics_evaluation_open_world_json_stats_df.columns:
    
    # Get the columns that consistently have only one value in them per session
    single_value_cols_set = set(logs_df.columns)
    for session_uuid, session_df in logs_df.groupby('session_uuid'):
        single_value_cols = set([col for col in session_df.columns if session_df[col].nunique() == 1])
        single_value_cols_set = single_value_cols_set.intersection(single_value_cols)
    print(single_value_cols_set)
    rows_list = []
    for session_uuid, session_df in logs_df[single_value_cols_set].dropna(axis='columns', how='all').groupby('session_uuid'):
        row_dict = {cn: session_df[cn].dropna().max() for cn in session_df.columns}
        rows_list.append(row_dict)
    single_value_cols_df = DataFrame(rows_list)
    print(single_value_cols_df.shape)
    
    on_columns = sorted(set(metrics_evaluation_open_world_json_stats_df.columns).intersection(set(single_value_cols_df.columns)))
    print(on_columns)
    metrics_evaluation_open_world_json_stats_df = metrics_evaluation_open_world_json_stats_df.merge(
        single_value_cols_df, how='left', on=on_columns
    )
    print(metrics_evaluation_open_world_json_stats_df.shape)
    print(single_value_cols_df.shape)
    
    # Save so you don't have to run it again
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    
    print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3538)

{'session_uuid', 'file_name', 'patient_demoted_health_time_remaining', 'csv_file_name', 'patient_engaged_health_level', 'player_location_left_hand_location', 'bag_access_location', 'injury_record_injury_treated', 'patient_record_health_level', 'voice_capture_command_description', 'player_location_right_hand_location', 'patient_demoted_health_level', 'injury_record_injury_treated_with_wrong_treatment', 'participant_id'}
(43, 14)
['participant_id', 'session_uuid']
(43, 3537)
(43, 14)
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv
(43, 3537)


In [7]:

metrics_evaluation_open_world_json_stats_df = nu.load_object('metrics_evaluation_open_world_json_stats_df')
if 'MedExp' not in metrics_evaluation_open_world_json_stats_df.columns:
    from pandas import read_excel
    file_path = '../data/xlsx/Metrics_Eval_Participant_data_for_BBAI.xlsx'
    participant_data_df = read_excel(file_path).rename(columns={'ParticipantID': 'participant_id', 'Date': 'participation_date'})
    for i in range(2):
        cn = 'Sim' + str(i+1)
        participant_data_df[cn] = participant_data_df[cn].map(lambda x: str(x)[:-1] if str(x).endswith('_') else x)
    print(participant_data_df.shape)
    sim1_list = participant_data_df.Sim1.tolist()
    assert len(sim1_list) == len(set(sim1_list)), "Sim1 has duplicates"
    sim2_list = participant_data_df.Sim2.tolist()
    assert len(sim2_list) == len(set(sim2_list)), "Sim2 has duplicates"
    assert set(sim1_list).intersection(set(sim2_list)) == set(), "There are duplicates between Sim1 and Sim2"
    
    # See if you can merge on participant_id
    on_columns = sorted(set(metrics_evaluation_open_world_json_stats_df.columns).intersection(set(participant_data_df.columns)))
    print(on_columns)
    print(metrics_evaluation_open_world_json_stats_df.shape)
    metrics_evaluation_open_world_json_stats_df = metrics_evaluation_open_world_json_stats_df.merge(
        participant_data_df, how='left', on=on_columns
    )
    print(participant_data_df.shape)
    print(metrics_evaluation_open_world_json_stats_df.shape)
    mask_series = (metrics_evaluation_open_world_json_stats_df.session_uuid == metrics_evaluation_open_world_json_stats_df.Sim1)
    mask_series |= (metrics_evaluation_open_world_json_stats_df.session_uuid == metrics_evaluation_open_world_json_stats_df.Sim2)
    print(
        f'Of the {metrics_evaluation_open_world_json_stats_df.shape[0]} JSON files, only'
        f' {metrics_evaluation_open_world_json_stats_df[mask_series].shape[0]} of them match with either Sim1 or Sim2.'
    )
    
    # Check if the various partipant id columns are inconsistent
    columns_list = [cn for cn in metrics_evaluation_open_world_json_stats_df.columns if cn.lower().startswith('participant') and cn.lower().endswith('id')]
    df = metrics_evaluation_open_world_json_stats_df[columns_list]
    for cn in columns_list:
        df[cn] = df[cn].map(lambda x: str(x).strip())
    mask_series = (df[columns_list[0]] != df[columns_list[1]])
    assert not mask_series.any(), "The various partipant id columns are inconsistent"
    
    # Save so you don't have to run it again
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)

(22, 27)
['participant_id']
(43, 3537)
(22, 27)
(43, 3563)
Of the 43 JSON files, only 14 of them match with either Sim1 or Sim2.
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv


In [8]:

# Remove any duplicate session IDs
mask_series = metrics_evaluation_open_world_json_stats_df.duplicated(subset='session_uuid')
if mask_series.any():
    display(metrics_evaluation_open_world_json_stats_df[mask_series].dropna(axis='columns', how='all')); raise
    metrics_evaluation_open_world_json_stats_df = metrics_evaluation_open_world_json_stats_df[~mask_series]
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    print(metrics_evaluation_open_world_json_stats_df.shape) # (60, 8)

In [9]:

# Fix the null file dates
if 'session_file_date' not in metrics_evaluation_open_world_json_stats_df.columns:
    metrics_evaluation_open_world_json_stats_df['session_file_date'] = pd.NaT
mask_series = metrics_evaluation_open_world_json_stats_df.session_file_date.isnull()
if mask_series.any():
    print(f'I have {mask_series.sum()} sessioms in my JSON Stats data frame without file dates.')
    for session_uuid, idx_df in metrics_evaluation_open_world_json_stats_df[mask_series].groupby('session_uuid'):
        
        # Get the whole session history
        mask_series = (logs_df.session_uuid == session_uuid)
        assert mask_series.any(), f"You're missing a Session ID {session_uuid} in logs_df"
        session_df = logs_df[mask_series]

        mask_series = ~session_df.event_time.isnull()
        assert mask_series.any(), f"Session ID {session_uuid} doesn't have an event times in logs_df"
        session_file_date = session_df[mask_series].event_time.min().date()
        metrics_evaluation_open_world_json_stats_df.loc[idx_df.index, 'session_file_date'] = session_file_date
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    mask_series = metrics_evaluation_open_world_json_stats_df.session_file_date.isnull()
    if mask_series.any():
        print(f'I still have {mask_series.sum()} sessioms in my JSON Stats data frame without file dates.')
        display(metrics_evaluation_open_world_json_stats_df[mask_series].dropna(axis='columns', how='all').T)

I have 43 sessioms in my JSON Stats data frame without file dates.
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv


In [10]:

# Add file start time
if ('file_start_time' not in metrics_evaluation_open_world_json_stats_df.columns) or ('file_stop_time' not in metrics_evaluation_open_world_json_stats_df.columns):
    sub_directory = '../data/logs'
    import os.path as osp
    if 'file_name' in metrics_evaluation_open_world_json_stats_df.columns:
        for file_name, idx_df in metrics_evaluation_open_world_json_stats_df.groupby('file_name'):
            
            # Construct the full path to the file
            file_path = osp.join(sub_directory, file_name)
            
            # Attempt to read CSV file using pandas
            try:
                import pandas as pd
                file_df = pd.read_csv(file_path, header=None, index_col=False)
            
            # If unsuccessful, try using a reader
            except:
                rows_list = []
                with open(file_path, 'r') as f:
                    import csv
                    reader = csv.reader(f, delimiter=',', quotechar='"')
                    for values_list in reader:
                        if (values_list[-1] == ''): values_list.pop(-1)
                        rows_list.append({i: v for i, v in enumerate(values_list)})
                from pandas import DataFrame
                file_df = DataFrame(rows_list)
            
            ts_series = to_datetime(file_df[2], infer_datetime_format=True)
            metrics_evaluation_open_world_json_stats_df.loc[idx_df.index, 'file_start_time'] = ts_series.min().to_pydatetime()
            metrics_evaluation_open_world_json_stats_df.loc[idx_df.index, 'file_stop_time'] = ts_series.max().to_pydatetime()
        print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3538)
        nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
        nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)

(43, 3566)
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv


In [11]:

# Get a sample with a clear count of responders
new_column_name = 'is_a_one_triage_file'
# if (new_column_name in metrics_evaluation_open_world_json_stats_df.columns):
    # metrics_evaluation_open_world_json_stats_df = metrics_evaluation_open_world_json_stats_df.drop(columns=new_column_name)
if (new_column_name not in metrics_evaluation_open_world_json_stats_df.columns):
    metrics_evaluation_open_world_json_stats_df[new_column_name] = False
    data_frames_dict = nu.load_data_frames(metrics_evaluation_open_world_scene_stats_df='')
    scene_stats_df = data_frames_dict['metrics_evaluation_open_world_scene_stats_df']
    
    # Assume a 1:1 correspondence between file name and UUID from the logs data frame build
    for session_uuid, session_df in metrics_evaluation_open_world_json_stats_df.groupby('session_uuid'):
        assert session_df.shape[0] == 1, "You've got duplicate session UUIDs"
        
        # Filter in the triage files in this UUID
        mask_series = (scene_stats_df.session_uuid == session_uuid) & (scene_stats_df.scene_type == 'Triage')
        
        # Get whether the file has only one triage run
        triage_scene_count = len(scene_stats_df[mask_series].groupby('scene_id').groups)
        is_a_one_triage_file = bool(triage_scene_count == 1)
        
        metrics_evaluation_open_world_json_stats_df.loc[session_df.index, new_column_name] = is_a_one_triage_file
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3539)

display(metrics_evaluation_open_world_json_stats_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_scene_stats_df.pkl.
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv
(43, 3567)


,record_count
is_a_one_triage_file,
False,2
True,41


In [12]:

new_column_name = 'is_in_registry'
if (new_column_name not in metrics_evaluation_open_world_json_stats_df.columns):
    metrics_evaluation_open_world_json_stats_df[new_column_name] = False
    
    # Store the results and show the new data frame shape
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df)
    print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3540)

display(metrics_evaluation_open_world_json_stats_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv
(43, 3568)


,record_count
is_in_registry,
False,43


In [13]:

# Fix the encounter_layout column based on the set of patients in the scene
new_column_name = 'encounter_layout'
desert_patients_list = ['Open World Marine 1 Female Root', 'Open World Marine 2 Male Root', 'Open World Civilian 1 Male Root', 'Open World Civilian 2 Female Root']
jungle_patients_list = ['Open World Marine 1 Male Root', 'Open World Marine 2 Female Root', 'Open World Marine 3 Male Root', 'Open World Marine 4 Male Root']
submarine_patients_list = ['Navy Soldier 1 Male Root', 'Navy Soldier 2 Male Root', 'Navy Soldier 3 Male Root', 'Navy Soldier 4 Female Root']
urban_patients_list = ['Marine 1 Male Root', 'Marine 2 Male Root', 'Marine 3 Male Root', 'Marine 4 Male Root', 'Civilian 1 Female Root']
for (session_uuid, scene_id), scene_df in logs_df.groupby(fu.scene_groupby_columns):
    for env_str in ['desert', 'jungle', 'submarine', 'urban']:
        patients_list = eval(f'{env_str}_patients_list')
        if all(map(lambda patient_id: patient_id in scene_df.patient_id.unique().tolist(), patients_list)):
            mask_series = (metrics_evaluation_open_world_json_stats_df.session_uuid == session_uuid)
            metrics_evaluation_open_world_json_stats_df.loc[mask_series, new_column_name] = env_str.title()
            nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df, verbose=False)
            nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df, verbose=False)
    
print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3541)
display(metrics_evaluation_open_world_json_stats_df.groupby(new_column_name, dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

(43, 3569)


,record_count
encounter_layout,
Desert,11
Jungle,11
Submarine,11
Urban,10



Please use the data attached for exploratory analyses in our open world scenarios. The document attached defines each of the variables (by column) and each row represents one human.

You should have TWO simulation data files for each participant (row) and they should be matched up by the UUIDs in columns 3 and 4 (they should be two different environments in case you have those in separate folders).

I would like you to look for any patterns in these data, especially related to the variables “ST_KDMA_Text” “ST_KDMA_Sim” “AD_KDMA_Text” “AD_KDMA_Sim”
Let me know if you have any questions.


In [14]:

# Add file start time
if ('in_sim1' not in metrics_evaluation_open_world_json_stats_df.columns) or ('in_sim2' not in metrics_evaluation_open_world_json_stats_df.columns):
    metrics_evaluation_open_world_json_stats_df['in_sim1'] = False
    metrics_evaluation_open_world_json_stats_df['in_sim2'] = False
    
    from pandas import read_excel
    file_path = '../data/xlsx/Metrics_Eval_Participant_data_for_BBAI.xlsx'
    participant_data_df = read_excel(file_path)
    
    mask_series = ~participant_data_df.Sim1.isnull() & (participant_data_df.Sim1 != '')
    sim1_session_uuids_list = [u[:-1] if u.endswith('_') else u for u in sorted(participant_data_df[mask_series].Sim1)]
    mask_series = ~participant_data_df.Sim2.isnull() & (participant_data_df.Sim2 != '')
    sim2_session_uuids_list = [u[:-1] if u.endswith('_') else u for u in sorted(participant_data_df[mask_series].Sim2)]
    assert set(sim1_session_uuids_list).intersection(set(sim2_session_uuids_list)) == set(), "The participant ID groupings are not distinct"
    
    mask_series = metrics_evaluation_open_world_json_stats_df.session_uuid.isin(sim1_session_uuids_list)
    metrics_evaluation_open_world_json_stats_df.loc[mask_series, 'in_sim1'] = True
    mask_series = metrics_evaluation_open_world_json_stats_df.session_uuid.isin(sim2_session_uuids_list)
    metrics_evaluation_open_world_json_stats_df.loc[mask_series, 'in_sim2'] = True
    
    nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df, verbose=True)
    nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df, verbose=True)
display(metrics_evaluation_open_world_json_stats_df.groupby('in_sim1', dropna=False).size().to_frame().rename(columns={0: 'record_count'}))
display(metrics_evaluation_open_world_json_stats_df.groupby('in_sim2', dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv


,record_count
in_sim1,
False,21
True,22


,record_count
in_sim2,
False,22
True,21


In [15]:

# Convert the participant ID columns to numeric
from pandas import to_numeric

columns_list = [cn for cn in metrics_evaluation_open_world_json_stats_df.columns if 'partici' in cn.lower()]
for cn in columns_list: metrics_evaluation_open_world_json_stats_df[cn] = to_numeric(metrics_evaluation_open_world_json_stats_df[cn], errors='coerce')
print(metrics_evaluation_open_world_json_stats_df.shape) # (43, 3570)
nu.store_objects(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df, verbose=True)
nu.save_data_frames(metrics_evaluation_open_world_json_stats_df=metrics_evaluation_open_world_json_stats_df, verbose=True)

(43, 3571)
Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/metrics_evaluation_open_world_json_stats_df.csv


In [16]:

raise

RuntimeError: No active exception to reraise


# Verify

In [ ]:

# See if you can merge on Sim1's session_uuid
sim1_df = metrics_evaluation_open_world_json_stats_df.merge(participant_data_df, left_on='session_uuid', right_on='Sim1', how='inner')
print(sim1_df.shape)
mask_series = (sim1_df.participant_id_x != sim1_df.participant_id_y)
assert mask_series.sum() == 0, f"{mask_series.sum()} files have mismatched Participant IDs in Sim1"

In [ ]:

# See if you can merge on Sim2's session_uuid
sim2_df = metrics_evaluation_open_world_json_stats_df.merge(participant_data_df, left_on='session_uuid', right_on='Sim2', how='inner')
print(sim2_df.shape)
mask_series = (sim2_df.participant_id_x != sim2_df.participant_id_y)
assert mask_series.sum() == 0, f"{mask_series.sum()} files have mismatched Participant IDs in Sim2"


# Visualize

In [ ]:

print(metrics_evaluation_open_world_json_stats_df.participant_id.nunique()) # 22

In [ ]:

# Show numeric columns
column_descriptions_df = nu.get_column_descriptions(participant_data_df)
mask_series = column_descriptions_df.dtype.isin(['float64', 'int64'])
columns_set = set(column_descriptions_df[mask_series].column_name).difference(set(['ParticipantID']))
list(columns_set)

In [ ]:

# Walk through the logs, getting only file names and session_uuids
from pandas import DataFrame, read_csv, concat
import os
from os import path as osp

important_columns_list = ['file_name', 'session_uuid']
file_names_and_session_uuids_df = DataFrame([], columns=important_columns_list)

# Iterate over the subdirectories, directories, and files in the logs folder
logs_folder = '../data/logs'
for sub_directory, directories_list, files_list in os.walk(logs_folder):
    
    # Create a data frame to store the data for the current subdirectory
    sub_directory_df = DataFrame([], columns=important_columns_list)
    
    # Iterate over the files in the current subdirectory
    for file_name in files_list:
        
        # If the file is a CSV file, merge it into the subdirectory data frame
        if file_name.endswith('.csv'):
            # sub_directory_df = fu.process_files(sub_directory_df, sub_directory, file_name, verbose=verbose)
            
            # Construct the full path to the file
            file_path = osp.join(sub_directory, file_name)
            
            # Attempt to read CSV file using pandas
            try: file_df = read_csv(file_path, header=None, index_col=False)
            
            # If unsuccessful, try using a reader
            except:
                rows_list = []
                with open(file_path, 'r') as f:
                    import csv
                    reader = csv.reader(f, delimiter=',', quotechar='"')
                    for values_list in reader:
                        if (values_list[-1] == ''): values_list.pop(-1)
                        rows_list.append({i: v for i, v in enumerate(values_list)})
                file_df = DataFrame(rows_list)
            
            # Ignore small files and return the subdirectory data frame unharmed
            if (file_df.shape[1] >= 16):
                
                # Add file name  to the data frame
                file_dir_suffix = osp.abspath(sub_directory).replace(osp.abspath(logs_folder) + os.sep, '')
                file_df['file_name'] = '/'.join(file_dir_suffix.split(os.sep)) + '/' + file_name
                
                # Name the global columns
                columns_list = ['action_type', 'action_tick', 'event_time', 'session_uuid']
                file_df.columns = columns_list + file_df.columns.tolist()[len(columns_list):]

                # Remove all but the file name and session columns
                file_df = file_df[important_columns_list].drop_duplicates()
                
                # Append the data frame for the current file to the data frame for the current subdirectory
                sub_directory_df = concat([sub_directory_df, file_df], axis='index')
    
    # Append the data frame for the current subdirectory to the main data frame
    file_names_and_session_uuids_df = concat([file_names_and_session_uuids_df, sub_directory_df], axis='index')
    
file_names_and_session_uuids_df = file_names_and_session_uuids_df.reset_index(drop=True)
mask_series = file_names_and_session_uuids_df.session_uuid.isin(sim_session_uuids_list)
df = file_names_and_session_uuids_df[mask_series]
print(df.shape) # (43, 3)
display(df.sample(5).T)

In [ ]:

import re

search_regex = re.compile('Urban', re.IGNORECASE)
columns_list = nu.get_regexed_columns(metrics_evaluation_open_world_json_stats_df, search_regex=search_regex)
df = nu.get_regexed_dataframe(metrics_evaluation_open_world_json_stats_df, columns_list, search_regex=search_regex)
df.sample(4)[columns_list].dropna(axis='columns', how='all').T

In [ ]:

columns_list = [
    'encounter_layout', 'configData_narrative_narrativeDescription', 'configData_scene', 'configData_scenarioData_name', 'configData_scenarioData_description'
]
for cn in columns_list: display(metrics_evaluation_open_world_json_stats_df.groupby(cn, dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

In [ ]:

def show_subgraph(sub_graph, suptitle='Within-function Function Calls', nodes_list_list=None, node_color='b', verbose=False):
    
    # Vertically separate the labels for easier readability
    layout_items = nx.spring_layout(sub_graph).items()
    left_lim, right_lim = -1500, 1500
    bottom_lim, top_lim = left_lim * nu.twitter_aspect_ratio, right_lim * nu.twitter_aspect_ratio
    rows_list = [{'node_name': node_name, 'layout_x': pos_array[0], 'layout_y': pos_array[1]} for node_name, pos_array in layout_items]
    df = DataFrame(rows_list).sort_values('layout_x')
    df['x_tick'] = [int(round(el)) for el in pd.cut(np.array([left_lim, right_lim]), len(sub_graph.nodes)+1, retbins=True)[1]][1:-1]
    df = df.sort_values('layout_y')
    df['y_tick'] = [int(round(el)) for el in pd.cut(np.array([bottom_lim, top_lim]), len(sub_graph.nodes)+1, retbins=True)[1]][1:-1]
    
    # Create the layout dictionary
    layout_dict = {}
    for row_index, row_series in df.iterrows():
        node_name = row_series.node_name
        layout_x = row_series.x_tick
        layout_y = row_series.y_tick
        layout_dict[node_name] = np.array([float(layout_x), float(layout_y)])
    
    # Draw the graph using the layout
    fig = plt.figure(figsize=(18, 7), facecolor='white')
    ax = fig.add_subplot(111)
    plt.axis('off')
    plt.xticks([], [])
    plt.yticks([], [])
    fig.suptitle(suptitle, fontsize=24)
    
    # Make the nodes the node_color
    if nodes_list_list is None:
        node_collection = nx.draw_networkx_nodes(
            G=sub_graph, pos=layout_dict, alpha=0.33, node_color=node_color.reshape(1, -1), node_size=150
        )
        edge_collection = nx.draw_networkx_edges(
            G=sub_graph, pos=layout_dict, alpha=0.25, width=[edge_tuple[2]['weight'] for edge_tuple in sub_graph.edges(data=True)]
        )
        labels_collection = nx.draw_networkx_labels(G=sub_graph, pos=layout_dict, font_size=10)
    
    # Color each nodes list differently
    else:
        if verbose: display(nodes_list_list)
        color_cycler = nu.get_color_cycler(len(nodes_list_list))
        for nodes_list, fcd in zip(nodes_list_list, color_cycler()):
            if verbose: display(fcd['color'])
            node_color = fcd['color'].reshape(1, -1)
            sub_subgraph = nx.subgraph(sub_graph, nodes_list)
            node_collection = nx.draw_networkx_nodes(G=sub_subgraph, pos=layout_dict, alpha=0.33, node_color=node_color, node_size=150)
            edge_collection = nx.draw_networkx_edges(G=sub_subgraph, pos=layout_dict, alpha=0.25)
            labels_collection = nx.draw_networkx_labels(G=sub_subgraph, pos=layout_dict, font_size=10)

In [ ]:

import itertools
import networkx as nx

# Create data structures to tally the nodes, edges, and weights
edge_tuple_dict = {}
for pair in itertools.combinations(columns_list, 2):
    df = metrics_evaluation_open_world_json_stats_df[list(pair)].drop_duplicates().sort_values(list(pair))
    # print()
    # print(*pair)
    eval_list = []
    for record_dict in df.dropna(axis='index', how='any').to_dict(orient='records'):
        values_list = list(record_dict.values())
        name_similarities_df = nu.check_for_typos(*[[v] for v in values_list])
        # print(*values_list)
        eval_list.append(1.0 if (values_list[0] in values_list[1]) else name_similarities_df.max_similarity.squeeze())
    similarity_measure = np.min(eval_list)
    edge_tuple_dict[pair] = similarity_measure#if similarity_measure > 0.08: 

# Create the directed graph
dg = nx.DiGraph()
dg.add_nodes_from(columns_list)
dg.add_weighted_edges_from([(k[0], k[1], 10*v) for k, v in edge_tuple_dict.items()])

In [ ]:

import matplotlib.pyplot as plt
show_subgraph(dg, suptitle='Encouter Layout Features', nodes_list_list=None, node_color=np.array([0.4, 0.4, 0.4, 1.0]), verbose=False)

In [ ]:

columns_list = ['configData_scene', 'configData_scenarioData_name', 'configData_scenarioData_description']
display(metrics_evaluation_open_world_json_stats_df[columns_list].drop_duplicates().sort_values(columns_list))
display(metrics_evaluation_open_world_json_stats_df.groupby(columns_list, dropna=False).size().to_frame().rename(columns={0: 'record_count'}))

In [ ]:

metrics_evaluation_open_world_json_stats_df.sample(5).T

In [ ]:

# Get the unique count of each column
for cn in sorted(metrics_evaluation_open_world_json_stats_df.columns): print(cn, metrics_evaluation_open_world_json_stats_df[cn].nunique())

In [ ]:

sorted(metrics_evaluation_open_world_json_stats_df.columns)

In [ ]:

display(metrics_evaluation_open_world_json_stats_df.groupby('responder_type').size().to_frame().rename(columns={0: 'record_count'}).sort_values(
    'record_count', ascending=False
).head(5))

In [ ]:

# Show all patient names
old_patient_names_set = set([
    'Adept Shooter Root', 'Adept Victim Root', 'Broken Bob Root', 'Broken Gloria Root', 'Broken Helga Root', 'Civilian 1 Female Root', 'Civilian 1 Root',
    'Civilian 2 Root', 'Intelligence Officer Root', 'Local Civilian with Internal Bleeding Root', 'Local Soldier 1 Root', 'Marine 1 Male Root', 'Marine 2 Male Root',
    'Marine 3 Male Root', 'Marine 4 Male Root', 'Marine with Leg Amputation Root', 'Marine with Narrative Root', 'NPC 1 Root', 'NPC 2 Root', 'NPC 3 Root',
    'NPC 4 Root', 'NPC Root', 'Navy Soldier 1 Male Root', 'Navy Soldier 2 Male Root', 'Navy Soldier 3 Male Root', 'Navy Soldier 4 Female Root',
    'Navy Solider 4 Female Root', 'Open World Civilian 1 Male Root', 'Open World Civilian 2 Female Root', 'Open World Marine 1 Female Root',
    'Open World Marine 1 Male Root', 'Open World Marine 2 Female Root', 'Open World Marine 2 Male Root', 'Open World Marine 3 Male Root',
    'Open World Marine 4 Male Root', 'Patient U Root', 'Patient V Root', 'Patient W Root', 'Patient X Root', 'Simulation Root', 'Tutorial Military Marine Root',
    'US Soldier 1 Root', 'bystander Root', 'electrician Root', 'patient U Root', 'patient V Root', 'patient W Root', 'patient X Root'
])
mask_series = ~logs_df.patient_id.isnull()
all_patient_names = sorted(logs_df[mask_series].patient_id.unique())
old_patient_names_set.symmetric_difference(set(all_patient_names))

In [ ]:

# Show all the responder types in the registry
df = data_frames_dict['first_responder_master_registry_file_stats_df']
mask_series = ~df.responder_type.isnull()
sorted(df[mask_series].responder_type.unique())

In [ ]:

# Show if you have any "file" columns in the logs data frame
sorted([cn for cn in logs_df.columns if 'file' in cn])